In [1]:
from simple_idml import idml
import xml.etree.ElementTree as ET

In [2]:
idml = idml.IDMLPackage("../IDMLlib/tests/assets/selling_secret.idml")

In [17]:
tree = ET.parse(idml.stories[-1])
print(ET.tostring(tree.getroot(), encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:Story xmlns:ns0="http://ns.adobe.com/AdobeInDesign/idml/1.0/packaging" DOMVersion="14.0">
	<Story AppliedNamedGrid="n" AppliedTOCStyle="n" IsEndnoteStory="false" Self="u19d7" StoryTitle="$ID/" TrackChanges="false" UserText="true">
		<StoryPreference FrameType="TextFrameType" OpticalMarginAlignment="false" OpticalMarginSize="12" StoryDirection="LeftToRightDirection" StoryOrientation="Horizontal" />
		<InCopyExportOption IncludeAllResources="false" IncludeGraphicProxies="true" />
		<ParagraphStyleRange AppliedParagraphStyle="ParagraphStyle/$ID/[No paragraph style]" Justification="CenterAlign" SpaceAfter="8">
			<Properties>
				<Mojikumi type="enumeration">Nothing</Mojikumi>
				<KinsokuSet type="enumeration">Nothing</KinsokuSet>
			</Properties>
			<CharacterStyleRange AppliedCharacterStyle="CharacterStyle/$ID/[No character style]" FillColor="Color/Word_R0_G0_B0" FontStyle="Italic" KerningMethod="$ID/Optical" PointSize="16">
				<Properties>
		

In [4]:
def get_alignment(args): 
    switcher = { 
        "CenterAlign": "text-align: center", 
        "RightAlign": "text-align: right", 
        "LeftAlign": "text-align: left", 
    } 
  
    return switcher.get(args, switcher["LeftAlign"]) 

def get_font_weight(args):
    switcher = { 
        "Bold": "font-weight: bold", 
        "Italics": "font-weight: italics", 
        "Normal": "font-weight: normal", 
    }
  
    return switcher.get(args, switcher["Normal"]) 

def get_font_size(args):
    return "font-size: " + args + "pt" if args else "font-size: 8pt"


# Parent styles

finalChild = ""
for story in idml.stories[:-1]:
    tree = ET.parse(story)
    root = tree.getroot()
    strContent = ''
    
    paragraphStyleRanges = root.iter("ParagraphStyleRange")
    
    for paragraphStyleRange in paragraphStyleRanges:
        parentStyle = ""
        alignment = paragraphStyleRange.attrib.get("Justification")
        alignment = get_alignment(alignment)
        parentStyle += "<p style='" + alignment + "'>"
        
        # Parent styles
        characterStyleRanges = paragraphStyleRange.iter('CharacterStyleRange')
        childStyle = ""
        for characterStyleRange in characterStyleRanges:
            fontStyle = characterStyleRange.attrib.get("FontStyle")
            fontStyle = get_font_weight(fontStyle)
            
            fontSize = characterStyleRange.attrib.get("PointSize")
            fontSize = get_font_size(fontSize)
            
            childrens = [i for i in characterStyleRange.iter()]
            
            for child in childrens:
                if child.tag == "Content":
                    childStyle += "<span style='" + fontStyle + ";"
                    childStyle += fontSize + ";" if fontSize else ""
                    childStyle += "'>" + child.text if child.text else "'>" 
                    childStyle += "</span>"
                
                if child.tag == "Br":
                    childStyle += "<br />"
                    
    finalChild += childStyle
                                 
strContent += parentStyle + finalChild + "</p>"

print(strContent)

<p style='text-align: center'><span style='font-weight: normal;font-size: 9pt;'></span><span style='font-weight: normal;font-size: 9pt;'></span><span style='font-weight: normal;font-size: 9pt;'></span><span style='font-weight: normal;font-size: 9pt;'></span><span style='font-weight: normal;font-size: 22pt;'>Book Title</span><span style='font-weight: bold;font-size: 16pt;'>Provided By Author Name</span><br /><span style='font-weight: bold;font-size: 16pt;'>•  •  •</span></p>
